# Creating a diagnostics package with CDP

In this notebook, we'll cover the steps needed to create a diagnostics package using the Community Diagnostics Package (CDP) via an example diagnostic package. For a basic diagnostics package, we'll cover how to:
* create metrics
* use them in your main script

Additional points covered will include:
* adding paramaters to your script
* adding command line functionality for the parameters
* optionally adding a viewer

## Overview of CDP

Add some overview?

## Creating metrics

In your project directory, create a folder that will contain all of your metrics. In our example, we'll use a folder titled `my_metrics`.

Our diagnostics will have two metrics `add` and `sub`, which will add and subtract two number respectively. Each metric must have a function called `compute()` and inherit from `CDPMetric`. Below is boilerplate code for creating a metric:

In [ ]:
from cdp.cdp_metric import CDPMetric

class MyMetric(CDPMetric):
    def compute():
        # put your code here

Create two files, `add.py` and `sub.py` in the `my_metrics` folder. Below is the actual code for the metrics:

In [ ]:
# add.py
from cdp.cdp_metric import CDPMetric

class Add(CDPMetric):
    def compute(num1, num2):
        return num1 + num2

In [ ]:
# sub.py
from cdp.cdp_metric import CDPMetric

class Sub(CDPMetric):
    def compute(num1, num2):
        return num1 - num2

So we can import these metrics in other scripts, we need to create an `__init__.py` file in the `my_metrics` folder. We can add the below code in `__init__.py` to make the syntax more clear:

In [ ]:
# my_metrics/__init__.py
import add
import sub

add = add.Add()
sub = sub.Sub()

## Creating your main script

Now all we need to do is import the metrics into your main script, which we'll call `driver.py`.

In [ ]:
# driver.py
from my_metrics import add, sub

print add(1, 1)
print sub(1, 1)

That's all that's needed to create a **very** basic diagnostics package using CDP. The next sections detail how to add more features such as parameters, commandline parsing, and adding in a viewer to view outputs.

## Adding support for parameters

Scientific software often consists of many variables, all of which might need to be changed for a given run. Software created with CDP allows for parameters to be handled in an organized way. 

Parameters are classes which inherit from `CDPParameter`. Below is boilerplate code.

In [ ]:
from cdp.cdp_parameter import CDPParameter

class MyParameter(CDPParameter):
    def check_values():
        # put your code here

The `check_values()` function is called automatically when a parameter is created. It allows for certain checks to be ran for variables in the parameter. 

In our example, the parameter will have two variables `num1` and `num2` which will be needed to checked. Below is the code that checks that both numbers are integers.

In [ ]:
# my_parameter.py
from cdp.cdp_parameter import CDPParameter

class MyParameter(CDPParameter):
    def __init__(self):
        self.num1 = None
        self.num2 = None
        
    def check_values(self):
        if not isinstance(self.num1, int):
            print('num1 needs to be an integer')
            quit()
        if not isinstance(self.num2, int):
            print('num2 needs to be an integer')
            quit()

### Using the parameter in the main script

Below is how one would use the parameter file in the main script.

In [ ]:
# driver.py
from my_metrics import add, sub
from my_parameter import MyParameter

parameter = MyParameter()
parameter.num1 = 1
parameter.num2 = 1
print add(parameter.num1, parameter.num2)
print sub(parameter.num1, parameter.num2)

# both lines below will be caught by check_values()
parameter.num1 = 'one'
parameter.num2 = 'two'

## Adding command line parsing

The above example of using parameters isn't the most useful. Adding command line parsing for parameters is a better solution and allows for running commands like this:

`driver.py --num1 1 --num2 1`

We can even run commands like this:

`driver.py -p parameter_file.py`

Where `parameter_file.py` is:

In [ ]:
# parameter_file.py

num1 = 1
num2 = 1

So to change the parameters, only the command line arguements (`--num1` and `--num2`) or `parameter_file.py` are needed to be changed instead of `driver.py`.

To do so, we need to create a parser, which inherits from `CDPParser`. Below is boilerplate code for a parser.

In [ ]:
# my_parser.py
from cdp.cdp_parser import CDPParser
from my_parameter import MyParameter

class MyParser(CDPParser):
    def __init__(self, *args, **kwargs):
        # creates parameters of type MyParameter
        super(MyParser, self).__init__(MyParameter, *args, **kwargs)

    def load_default_args(self):
        # this has '-p' and '--parameter' reserved
        super(MyParser, self).load_default_args()

        # put your arguments below

Below is the parser for this example project. Note that adding an argument to parser in CDP is analogous to how one would would [add arguments using argparse](https://docs.python.org/2.7/library/argparse.html#adding-arguments).

In [ ]:
# my_parser.py
from cdp.cdp_parser import CDPParser
from my_parameter import MyParameter

class MyParser(CDPParser):
    def __init__(self, *args, **kwargs):
        # creates parameters of type MyParameter
        super(MyParser, self).__init__(MyParameter, *args, **kwargs)

    def load_default_args(self):
        # this has '-p' and '--parameter' reserved
        super(MyParser, self).load_default_args()

        self.add_argument(
            '--num1',
            type=int,
            dest='num1',
            help='First num',
            required=False)

        self.add_argument(
            '--num2',
            type=int,
            dest='num2',
            help='Second num',
            required=False)

### Editing the main script to use the parser

All we need to do is import the MyParser class we just made and use it like so.

In [ ]:
# driver.py
from my_metrics import add, sub
from my_parser import MyParser

parser = MyParser()
parameter = parser.get_parameter()
print add(parameter.num1, parameter.num2)
print sub(parameter.num1, parameter.num2)

Again, now we can run commands like:

`driver.py --num1 1 --num2 2`

`driver.py --num1 14 --num2 12`

`driver.py -p parameter_file.py`

## Adding in the viewer

If you have scripts or output images that you want to visualize via a webpage, read the [Using the CDP Viewer](https://github.com/UV-CDAT/cdp/blob/master/jupyter/using-the-cdp-viewer.ipynb) guide.